In [1]:
import sys

sys.path.append("../model")

from Experiment import Experiment
from samplers.NFSampler import NFSampler
from samplers.SpikeSlabSampler import SpikeSlabSampler
from samplers.RandomWalkSampler import RandomWalkSampler
from searchers.GNNExplainerSearcher import GNNExplainerSearcher
from searchers.GreedySearcher import GreedySearcher

import torch
from torch_geometric.utils import k_hop_subgraph

from julia import Main
from julia import Turing
from julia import Base
from julia import StatsBase

Main.include("rw_model.jl")

[ Info: [Turing]: progress logging is disabled globally


<PyCall.jlwrap adj>

[ Info: [AdvancedVI]: global PROGRESS is set as false


In [2]:
experiment = Experiment("syn3-full-verified", "..")
experiment.train_base_model()

Loading syn3 dataset


2022-02-27 17:34:29.303766: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/runtime/opt/julia/1.7.1/lib:/gpfs/runtime/opt/gcc/8.3/lib64:/gpfs/runtime/opt/python/3.7.4/lib:/gpfs/runtime/opt/R/3.4.3_mkl/lib64/R/lib:/gpfs/runtime/opt/intel/2017.0/lib/intel64:/gpfs/runtime/opt/intel/2017.0/mkl/lib/intel64:/gpfs/runtime/opt/java/8u111/jre/lib/amd64
2022-02-27 17:34:29.303790: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
N = 529
subset, edge_index_adj, mapping, edge_mask_hard = k_hop_subgraph(
            N, 3, experiment.edge_index, relabel_nodes=True)

total_mapping = {k: i for k, i in enumerate(subset.tolist())}
x_adj = experiment.x[subset]

N = edge_index_adj.size(1)

with torch.no_grad():
    y = experiment.model(x_adj, edge_index_adj).exp()[mapping[0]]

In [5]:
model = Main.rw_model(edge_index_adj.numpy(), edge_index_adj.max().item(), 0.1, mapping.item(), x_adj, y.numpy(), experiment.model)
chains = Turing.sample(model, Turing.PG(100), 1000)

[ Info: Set{Graphs.SimpleGraphs.SimpleEdge}()


RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching !(::Nothing)
Closest candidates are:
  !(!Matched::Function) at /gpfs/rt/7.2/opt/julia/1.7.1/share/julia/base/operators.jl:1117
  !(!Matched::Bool) at /gpfs/rt/7.2/opt/julia/1.7.1/share/julia/base/bool.jl:35
  !(!Matched::Missing) at /gpfs/rt/7.2/opt/julia/1.7.1/share/julia/base/missing.jl:101
  ...
Stacktrace:
  [1] consume(ttask::TapedTask)
    @ Libtask ~/.julia/packages/Libtask/zSiN2/src/tapedtask.jl:138
  [2] advance!(trace::AdvancedPS.Trace{Turing.Essential.TracedModel{AbstractMCMC.AbstractSampler, DynamicPPL.AbstractVarInfo, DynamicPPL.Model, Tuple}})
    @ Turing.Essential ~/.julia/packages/Turing/rl6ku/src/essential/container.jl:50
  [3] reweight!(pc::AdvancedPS.ParticleContainer{AdvancedPS.Trace{Turing.Essential.TracedModel{AbstractMCMC.AbstractSampler, DynamicPPL.AbstractVarInfo, DynamicPPL.Model, Tuple}}})
    @ AdvancedPS ~/.julia/packages/AdvancedPS/DVEla/src/conta

In [ ]:
from julia import StatsBase
import numpy as np

arr = Base.Array(chains)
arr.mean(axis=0)

array([0.103, 0.094, 0.103, 0.095, 0.102, 0.09 , 0.114, 0.104, 0.104,
       0.112, 0.118, 0.095, 0.092, 0.115, 0.113, 0.107, 0.102, 0.109,
       0.099, 0.096, 0.103, 0.105, 0.102, 0.127, 0.094, 0.097, 0.092,
       0.105, 0.088, 0.09 , 0.098, 0.104, 0.099, 0.098, 1.291])

In [ ]:
arr

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 2.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])